In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
posts = pd.read_csv('../data/raw_data/posts.csv')
users = pd.read_csv('../data/raw_data/users.csv')

In [3]:
users.sample(5)

,uid,first_name,last_name,sex,is_leader
21196,228795784,Ruslan ...,Zeinalov,2,0
3406,164117746,Димка ...,Саранцев,2,0
25195,10629862,Анна ...,Якобсон,1,0
28242,256404793,Qis-Beach ...,Club,2,0
13454,1748226,Денис ...,Зайцев,2,0


In [4]:
posts.sample(5)

,from_id,owner_id,text,comments,likes,reposts
1185082,138247,3855514,прошла неделя....)))),2,0,0
3010642,2501214,2501214,Ik wil een kus een ik wil je zoenen... Hahah S...,0,0,0
837362,4894406,4894406,Я НЕ ИЩУ СВОЮ ВТОРУЮ ПОЛОВИНКУ..ПРОСТО Я РОДИЛ...,0,0,0
2407680,6376837,5305976,"Ксюш,спасибо за все!)было оооочень круто)еще п...",1,1,0
3693849,7523812,7523812,"Доброе утро Москва рабочая! До скорой встречи,...",0,13,0


In [5]:
valid_users = np.unique(users['uid'])

In [6]:
posts = posts[posts['from_id'].isin(valid_users)]
posts['text'] = posts['text'].astype('str')
data = posts.groupby(['from_id'], sort=True).agg({'text': ' '.join,
                                                  'comments': 'sum',
                                                  'likes': 'sum',
                                                  'reposts': 'sum'})

In [7]:
users.index = users.uid
users.drop('uid', axis=1, inplace=True)

In [8]:
data['is_leader'] = users.loc[data.index]['is_leader']

In [9]:
X = data.drop('is_leader', axis=1)
Y = data['is_leader'].values

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer()
model = LogisticRegression()


model.fit(tfidf.fit_transform(X['text']), Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
preds = model.predict(tfidf.transform(X['text']))

In [12]:
submission = pd.DataFrame({'is_leader':preds}, index=X.index)

submission.to_csv('../data/submission/train_submission.csv')